In [1]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [2]:
import openai
import math
import pandas as pd
import psycopg2
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import openai
from io import StringIO, BytesIO
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

Mock data and models:

In [3]:
class MockModelRepo:
    def __init__(self):
        self.models = [
            {"id": 1, "name": "Linear Regression 1", "description": "A model suitable for text data."},
            {"id": 2, "name": "LInear Regression 2", "description": "Optimized for image recognition tasks."},
        ]

    def get_all_model_metadata(self):
        return self.models

model_repo.py file:

In [4]:
query = "" #modify this to get the actual query from the user
repsonse = ""
llm_api_key = "sk-proj-8W1tHA2pEDS5VTVw0rDHT3BlbkFJN0mGTTl6JbkhUqBJ81Wo" # API key from OPENAI

#function to calculate the confidence of gpt based on the logprobabilities of all tokens
def calculate_confidence(logprobs):
    # Calculate probabilities from log probabilities and average them
    probabilities = [math.exp(logprob['logprob']) for logprob in logprobs]
    average_probability = sum(probabilities) / len(probabilities)
    # Convert to a percentage confidence score
    confidence_score = average_probability * 100
    return confidence_score

# Function to determine if RAG is needed or the LLM can answer the question as is
def is_rag_needed(query, llm_api_key):
    # Set up the API connection
    openai.api_key = llm_api_key

    # Call the LLM with the user's query
    try:
        # Make a request to OpenAI's GPT-4 with log probabilities enabled
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Specify the GPT-4 model
            messages=[{"role": "user", "content": query}],
            max_tokens=150,
            n=1,
            stop=None,
            temperature=0.7,
            logprobs=True  # Enable log probabilities for each token
        )
        # Analyze the LLM's response
        # The confidence score is how confident the LLM is that it can answer the query without RAG
        # Here we use the sum of the top log probabilities as a simple confidence measure
        logprobs_data = response['choices'][0]['logprobs']['content']
        # Calculate confidence score
        confidence_score = calculate_confidence(logprobs_data)
        print(confidence_score)

        # Define a confidence threshold to decide if RAG is needed
        confidence_threshold = 80  # will need to test with and modify this

        # If the confidence score is below the threshold, RAG is needed
        if confidence_score < confidence_threshold:
            return True
        else:
            return False
    except Exception as e:
        return {"error": str(e)}  # Return an error as a dictionary



#gets the descriptions of models in the model repo
def fetch_model_descriptions(model_repo):
    """
    Fetch metadata for all models in the repository.
    Assume that a function to retrieve model descriptions from the repo exists.
    """
    # This should return a dictionary or list of model metadata
    return model_repo.get_all_model_metadata()

#helper function querying LLM to get best model from repo
def query_llm_for_model_selection(query, context, model_descriptions):
    """
    This function uses the LLM to decide which model to select based on the user query,
    additional context, and descriptions of available models.
    """
    prompt = f"Find the best model from the model repo based on the following: Query: {query}\nContext: {context}\nModel Descriptions: {model_descriptions}\nSelect the best model:"

    response = openai.ChatCompletion.create(
        model="gpt-4",  # Specify the GPT-4 model
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0
    )

    # Extract the suggested model from the LLM's response
    #suggested_model = response.choices[0].text.strip()
    suggested_model = response['choices'][0]['message']['content'].strip()

    return suggested_model

#function querying LLM to get best model from repo
def select_model_for_query(query, model_repo):
    """
    Main function to select the best model for a given user query.
    """
    # Step 1: Fetch all the data related to the query (context)
    context = "a list of descriptions of documents from elasticsearch"  # This should contain the description of text chunks from elasticsearch to pass as context

    # Step 2: Fetch model descriptions
    model_descriptions = fetch_model_descriptions(model_repo)

    # Step 3: Use the LLM to suggest the best model
    best_model_response = query_llm_for_model_selection(query, context, model_descriptions)

    #need to change this to return the chosen model object not a string

    return best_model_response



#once data is retrieved from structured databases and elasticsearch, transform into a common type:
def process_data(data, data_format, model_metadata, api_key):
    # Read the data based on format
    if data_format == 'csv':
        df = pd.read_csv(StringIO(data))
    elif data_format == 'xlsx' or data_format == 'xls':
        df = pd.read_excel(BytesIO(data))
    elif data_format == 'sql':
        # Assuming 'data' is a SQL query already executed and returned as a string
        df = pd.read_sql(data)
    elif data_format == 'parquet':
        df = pd.read_parquet(BytesIO(data))
    elif data_format == 'json':
        df = pd.read_json(StringIO(data))
    elif data_format == 'xml':
        df = pd.read_xml(StringIO(data))

    # Use LLM to identify columns relevant to the model's inputs
    openai.api_key = api_key
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=f"Identify columns and data transformation needed for model with metadata: {model_metadata} in data: {df.head().to_json()}",
        max_tokens=150
    )
    preprocessing_instructions = response.choices[0].text.strip()

    # Apply transformations based on LLM output or rule-based programming
    # Example: Transforming data as per identified instructions
    #The important columns can be stored in variables, then using a set of predefined functions we can columsn to proper output
    #Or we can make the LLM do it all (need to experiment on what causes more time, latency...)
    exec(preprocessing_instructions, {'df': df, 'pd': pd})  # Caution: Using exec() safely requires careful handling.

    return df

# Pre-trained model (hypothetically loaded)
def predict(model, data):
    # This function would contain code to make predictions using the loaded model
    return model.predict(data)

def main(api_key, model, data, data_format, model_metadata):
    # Process the data
    processed_data = process_data(data, data_format, model_metadata, api_key)

    # Assuming 'model' is already trained and loaded elsewhere in your application
    predictions = predict(model, processed_data)

    # Output or use the predictions
    print("Predictions:", predictions)




In [5]:
# Example usage
query = "how many burgers can i eat in an hour"
llm_api_key = "sk-proj-8W1tHA2pEDS5VTVw0rDHT3BlbkFJN0mGTTl6JbkhUqBJ81Wo" #
model_repo = MockModelRepo  # replace with our actual model repo interface, this will likely link to a small database of models and their descriptions
needs_rag = is_rag_needed(query, llm_api_key)
data = "/content/Height vs Age.xlsx"  # This would be actual data or a path
data_format = 'xlsx'  # For demonstration, let's say the data is in xlsx format
print("RAG Needed?:", needs_rag)

64.04097534221181
RAG Needed?: True


In [6]:
class MockModelRepo:
    def __init__(self):
        self.models = [
            {"id": 1, "name": "Linear Regression 1", "description": "A model suitable for text data."},
            {"id": 2, "name": "LInear Regression 2", "description": "Optimized for image recognition tasks."},
        ]

    def get_all_model_metadata(self):
        return self.models
    def get_model_metadata(self, model_id):
        return self.models.get(model_id, {})

def fetch_model_descriptions(model_repo):
    return model_repo.get_all_model_metadata()

# Create an instance of MockModelRepo
model_repo_instance = MockModelRepo()

# Use the instance in your function call
best_model = select_model_for_query(query, model_repo_instance)
print("Selected Model:", best_model)

Selected Model: Based on the provided descriptions, the best model for this task would be 'Linear Regression 1' as it is suitable for text data. The task involves processing a text query and context, not image recognition.


In [7]:
model_repo = MockModelRepo()
model_metadata = model_repo.get_model_metadata(best_model.id)
main(llm_api_key, best_model, data, data_format, model_metadata)

AttributeError: 'str' object has no attribute 'id'

Usage: (needs to fix this part)